In [ ]:
import os
import pandas as pd
import numpy as np
import scipy.io
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, 
    Reshape, multiply, add, Activation, Lambda, Concatenate, Dropout
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
print("TensorFlow Version:", tf.__version__)

# --- Configuration ---
DATASET_PATH = 'D:/challa' 
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.0005

# --- 1. Data Loading ---
def parse_filenames_to_dataframe(dataset_path):
    records = []
    expanded_path = os.path.expanduser(dataset_path)
    print(f"Scanning main directory: {expanded_path}...")

    if not os.path.isdir(expanded_path):
        print(f"Error: Directory not found. Check DATASET_PATH.")
        return pd.DataFrame()

    bearing_dirs = [d for d in os.listdir(expanded_path) 
                    if os.path.isdir(os.path.join(expanded_path, d)) and not d.startswith('.')]
    print(f"Found bearing type folders: {bearing_dirs}")
    
    for bearing_type in bearing_dirs:
        bearing_path = os.path.join(expanded_path, bearing_type)
        for root, _, files in os.walk(bearing_path):
            for file in files:
                if file.endswith('.mat'):
                    file_parts = file.replace('.mat', '').split('_')
                    if len(file_parts) == 5:
                        speed = int(file_parts[4])
                        records.append({
                            'filepath': os.path.join(root, file),
                            'filename': file,
                            'speed': speed
                        })
    
    df = pd.DataFrame(records)
    print(f"Scan complete. Found {len(df)} total .mat files.")
    return df

# --- 2. Label Processing ---
def process_labels(df):
    print("\nProcessing labels for joint classification and domain adaptation...")
    
    df['joint_code'] = df['filename'].apply(lambda f: f.split('_')[0] + '_' + f.split('_')[1])
    df['joint_label'], joint_classes = pd.factorize(df['joint_code'])
    
    df['domain_label'], domain_classes = pd.factorize(df['speed'])
    
    print("Label processing complete.")
    print(f"Found {len(joint_classes)} unique joint conditions.")
    print(f"Found {len(domain_classes)} unique domains (speeds).")
    
    return df, list(domain_classes), list(joint_classes)

# --- 3. Data Pipeline ---
def load_and_process_file(filepath, joint_label, domain_label):
    mat_contents = scipy.io.loadmat(filepath.numpy())
    spectrogram_data = mat_contents['Spectrogram'].astype(np.float32)
    
    if np.max(spectrogram_data) > 0:
        spectrogram_data = spectrogram_data / np.max(spectrogram_data)
    
    images = np.expand_dims(spectrogram_data, axis=-1)
    
    num_samples = images.shape[0]
    
    joint_labels = to_categorical([joint_label] * num_samples, num_classes=32)
    domain_labels = to_categorical([domain_label] * num_samples, num_classes=6)
    
    return images, joint_labels, domain_labels

def tf_load_and_process(filepath, labels):
    images, j_labels, d_labels = tf.py_function(
        func=load_and_process_file,
        inp=[filepath, labels['joint'], labels['domain']],
        Tout=(tf.float32, tf.float32, tf.float32)
    )
    
    images.set_shape([None, IMG_SIZE, IMG_SIZE, 1])
    j_labels.set_shape([None, 32])
    d_labels.set_shape([None, 6])
    
    labels_dict = {
        "joint_output": j_labels,
        "domain_output": d_labels
    }
    
    return tf.data.Dataset.from_tensor_slices((images, labels_dict))

# --- 4. Model Architecture ---
@tf.custom_gradient
def grad_reverse(x, lambda_val=1.0):
    y = tf.identity(x)
    def custom_grad(dy):
        return -lambda_val * dy, None
    return y, custom_grad

class GradientReversal(tf.keras.layers.Layer):
    def __init__(self, lambda_val=1.0):
        super().__init__()
        self.lambda_val = lambda_val

    def call(self, x):
        return grad_reverse(x, self.lambda_val)

def cbam_module(x, ratio=8):
    channels = x.shape[-1]
    avg_pool = GlobalAveragePooling2D()(x)
    max_pool = tf.keras.layers.GlobalMaxPooling2D()(x)
    shared_dense_one = Dense(channels // ratio, activation='relu')
    shared_dense_two = Dense(channels)
    avg_out = shared_dense_two(shared_dense_one(avg_pool))
    max_out = shared_dense_two(shared_dense_one(max_pool))
    channel_attention = Activation('sigmoid')(add([avg_out, max_out]))
    channel_attention = Reshape((1, 1, channels))(channel_attention)
    x_channel = multiply([x, channel_attention])
    avg_pool_spatial = Lambda(lambda y: tf.reduce_mean(y, axis=3, keepdims=True))(x_channel)
    max_pool_spatial = Lambda(lambda y: tf.reduce_max(y, axis=3, keepdims=True))(x_channel)
    concat = Concatenate(axis=3)([avg_pool_spatial, max_pool_spatial])
    spatial_attention = Conv2D(1, (7, 7), padding='same', activation='sigmoid')(concat)
    return multiply([x_channel, spatial_attention])

def build_efficient_model(input_shape=(128, 128, 1)):
    inputs = Input(shape=input_shape, name='input')
    
    # Add L2 regularization to the convolutional layers
    x = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(x)
    x = MaxPooling2D((2, 2))(x)
    x = cbam_module(x, ratio=8)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(x)
    x = MaxPooling2D((2, 2))(x)
    
    features = Flatten()(x)
    features = Dropout(0.5)(features)
    
    # Add L2 regularization to the dense layers
    joint_head = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(features)
    joint_head = Dropout(0.3)(joint_head)
    joint_output = Dense(32, activation='softmax', name='joint_output')(joint_head)
    
    domain_features = GradientReversal(lambda_val=0.5)(features)
    # --- SUGGESTED ADDITION IS HERE ---
    domain_head = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(domain_features)
    domain_output = Dense(6, activation='softmax', name='domain_output')(domain_head)
    
    model = Model(inputs=inputs, outputs=[joint_output, domain_output])
    return model

# --- 5. Main Execution ---
if __name__ == "__main__":
    master_df = parse_filenames_to_dataframe(DATASET_PATH)
    
    if not master_df.empty:
        master_df, domain_classes, joint_classes = process_labels(master_df)
        
        train_df, val_df = train_test_split(
            master_df, test_size=0.2, random_state=42, stratify=master_df['joint_label']
        )
        print(f"\nTrain samples: {len(train_df)}")
        print(f"Validation samples: {len(val_df)}")
        
        # Create a dataset of filepaths and labels
        train_labels_ds = tf.data.Dataset.from_tensor_slices({
            'joint': train_df['joint_label'].values, 
            'domain': train_df['domain_label'].values
        })
        train_filepaths_ds = tf.data.Dataset.from_tensor_slices(train_df['filepath'].values)
        train_ds = tf.data.Dataset.zip((train_filepaths_ds, train_labels_ds))
        
        # Build the final, correct training pipeline
        train_dataset = train_ds.shuffle(buffer_size=len(train_df))
        train_dataset = train_dataset.flat_map(tf_load_and_process)
        train_dataset = train_dataset.shuffle(buffer_size=10000)
        train_dataset = train_dataset.repeat() # <-- FIX IS HERE
        train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
        
        # Build the final validation pipeline
        val_labels_ds = tf.data.Dataset.from_tensor_slices({
            'joint': val_df['joint_label'].values, 
            'domain': val_df['domain_label'].values
        })
        val_filepaths_ds = tf.data.Dataset.from_tensor_slices(val_df['filepath'].values)
        val_ds = tf.data.Dataset.zip((val_filepaths_ds, val_labels_ds))
        
        val_dataset = val_ds.flat_map(tf_load_and_process)
        val_dataset = val_dataset.repeat() # <-- AND FIX IS HERE
        val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
        
        print("\n Data pipelines created successfully.")
        
        model = build_efficient_model() 
        model.compile(
            optimizer=Adam(learning_rate=LEARNING_RATE),
            loss={'joint_output': 'categorical_crossentropy', 'domain_output': 'categorical_crossentropy'},
            loss_weights={'joint_output': 1.0, 'domain_output': 0.1},
            metrics={'joint_output': 'accuracy', 'domain_output': 'accuracy'}
        )
        model.summary()
        
        callbacks = [
            EarlyStopping(monitor='val_joint_output_accuracy', mode='max', patience=10, restore_best_weights=True, verbose=1),
            ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)
        ]
        
        # Calculate steps per epoch
        train_steps = len(train_df) * 78 // BATCH_SIZE
        val_steps = len(val_df) * 78 // BATCH_SIZE
        print(f"\nTraining steps per epoch: {train_steps}")
        print(f"Validation steps per epoch: {val_steps}")
        
        # Train the model
        print("\n--- Starting Model Training ---")
        history = model.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=EPOCHS,
            steps_per_epoch=train_steps,
            validation_steps=val_steps,
            callbacks=callbacks,
            verbose=1
        )
        
        print("\n Model training complete!")
        
        # Final Evaluation
        print("\n--- Final Model Evaluation ---")
        results = model.evaluate(val_dataset, steps=val_steps, verbose=0)
        
        print("\nFinal Performance Metrics:")
        print(f"Total Loss: {results[0]:.4f}")
        print(f"Joint Classification Loss: {results[1]:.4f}")
        print(f"Domain Head Loss: {results[2]:.4f}")
        print(f"Domain Head Accuracy: {results[3]*100:.2f}%")
        print(f"Joint Classification Accuracy: {results[4]*100:.2f}%")
        
        model.save('efficient_mt_cnn_da_model.h5')
        print("\nModel saved as 'efficient_mt_cnn_da_model.h5'")